# PROGRESS LABEL & EXPORT TO CSV IS WORKING
* DO NOT EDIT THIS

In [ ]:
#### DO NOT EDIT THIS CODE ####


# Import Dependencies
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text = "Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        if row not in sql_server_data:
            differences.append(row)

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline = '') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()  
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main even loop
window.mainloop()

# Edit anything placed below this

In [ ]:
#### DO NOT EDIT THIS CODE ####


# Import Dependencies
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        if row not in sql_server_data:
            differences.append((row, "Row does not exist in SQL Server table"))
        else:
            index = sql_server_data.index(row)
            if snowflake_data[index] != row:
                differences.append((row, "Value or data type difference"))

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            header = ["Data"] + ["Explanation"]
            csv_writer.writerow(header)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()
    
# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()  
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main even loop
window.mainloop()

In [ ]:
#### DO NOT EDIT THIS CODE ####


# Import Dependencies
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Get the primary key column
    primary_key_column = 'id'  # Replace with the actual primary key column name

    # Create dictionaries for easier lookup
    snowflake_dict = {row[primary_key_column]: row for row in snowflake_data}
    sql_server_dict = {row[primary_key_column]: row for row in sql_server_data}

    # Compare data between the selected tables
    differences = []
    snowflake_keys = set(snowflake_dict.keys())
    sql_server_keys = set(sql_server_dict.keys())
    common_keys = snowflake_keys.intersection(sql_server_keys)

    for key in common_keys:
        if snowflake_dict[key] != sql_server_dict[key]:
            differences.append((snowflake_dict[key], sql_server_dict[key], "Value or data type difference"))

    # Find rows that exist only in Snowflake table
    snowflake_only_keys = snowflake_keys - common_keys
    for key in snowflake_only_keys:
        differences.append((snowflake_dict[key], None, "Row does not exist in SQL Server table"))

    # Find rows that exist only in SQL Server table
    sql_server_only_keys = sql_server_keys - common_keys
    for key in sql_server_only_keys:
        differences.append((None, sql_server_dict[key], "Row does not exist in Snowflake table"))

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            header = ["Snowflake Data"] + ["SQL Server Data"] + ["Explanation"]
            csv_writer.writerow(header)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()


# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()  
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main even loop
window.mainloop()

# Code below is trying to use the primary keys to sort the tables

In [ ]:
# Import Dependencies
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Get the primary key column for Snowflake table
def get_snowflake_primary_key(snowflake_table):
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SHOW COLUMNS IN {snowflake_table}")
    columns = snowflake_cursor.fetchall()
    for column in columns:
        if column[6] == 'Y':  # Check if column is primary key
            return column[1]  # Return column name
    return None

# Get the primary key column for SQL Server table
def get_sql_server_primary_key(sql_server_table):
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE WHERE OBJECT_NAME(OBJECT_ID) = '{sql_server_table}'")
    row = cursor.fetchone()
    if row:
        return row[0]  # Return column name
    return None

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Get the primary key columns
    snowflake_primary_key = get_snowflake_primary_key(snowflake_table)
    sql_server_primary_key = get_sql_server_primary_key(sql_server_table)

    if snowflake_primary_key is None:
        print("Unable to find primary key for Snowflake table")
        return

    if sql_server_primary_key is None:
        print("Unable to find primary key for SQL Server table")
        return

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        # Extract the primary key value from the Snowflake row
        snowflake_primary_key_value = row[snowflake_cursor.description.index((snowflake_primary_key, None, None, None, None, None, None))]

        # Find matching row in SQL Server data based on the primary key value
        matching_rows = [sql_row for sql_row in sql_server_data if sql_row[sql_server_cursor.description.index((sql_server_primary_key, None, None, None, None, None, None))] == snowflake_primary_key_value]

        if len(matching_rows) == 0:
            differences.append(list(row) + ['Row does not exist in SQL Server'])
        else:
            # Check for value differences in non-primary key columns
            sql_server_row = matching_rows[0]
            for i in range(len(row)):
                if i != snowflake_cursor.description.index((snowflake_primary_key, None, None, None, None, None, None)):
                    if row[i] != sql_server_row[i]:
                        differences.append(list(row) + ['Value difference in column'])
                        break

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow([column[0] for column in snowflake_cursor.description] + ['Explanation'])
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main even loop
window.mainloop()


In [ ]:
# Import Dependencies
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text = "Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        if row not in sql_server_data:
            differences.append(row)

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline = '') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()  
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main even loop
window.mainloop()

# Trying to get the autocomplete to work

In [ ]:
## THIS MAKES THE SQL SERVER TABLES DROPDOWN WEIRD ##

import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 
from ttkwidgets.autocomplete import AutocompleteCombobox

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text = "Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        if row not in sql_server_data:
            differences.append(row)

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline = '') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = AutocompleteCombobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()  
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = AutocompleteCombobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main event loop
window.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role

# Autocomplete feature for the Combobox
def autocomplete_combobox(event):
    combobox = event.widget
    combobox_values = combobox["values"]

    typed_text = combobox.get()
    matching_values = [value for value in combobox_values if typed_text.lower() in value.lower()]

    combobox.configure(values=matching_values)

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        if row not in sql_server_data:
            differences.append(row)

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)

# Bind the autocompletion function to the key release event of the SQL Server table dropdown
sql_server_table_dropdown.bind("<KeyRelease>", autocomplete_combobox)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main event loop
window.mainloop()


# Try this new script
* Feel free to edit this one

In [ ]:
import tkinter as tk
import snowflake.connector as sf
import pyodbc
from config import ssms_connector, account, user, authenticator, warehouse, database, role

# Create a Tkinter GUI window
window = tk.Tk()
window.title("Table Comparison")
window.geometry("500x400")

# Create a text box to display the differences
text_box = tk.Text(window, height=20, width=60)
text_box.pack()

def compare_tables():
    # Clear the text box
    text_box.delete(1.0, tk.END)

    # Connect to Snowflake
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)

    # Connect to SQL Server
    sql_server_conn = pyodbc.connect(ssms_connector)

    # Get Snowflake table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SHOW TABLES IN {database}.{SNOWFLAKE_SCHEMA}")
    snowflake_tables = [row[1] for row in snowflake_cursor]

    # Get SQL Server table names
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
    sql_server_tables = [row[0] for row in sql_server_cursor]

    # Compare tables
    for table in set(snowflake_tables + sql_server_tables):
        if table in snowflake_tables and table in sql_server_tables:
            # Compare table columns
            snowflake_cursor.execute(f"DESCRIBE TABLE {table}")
            snowflake_columns = [row[1] for row in snowflake_cursor]
            sql_server_cursor.execute(f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table}'")
            sql_server_columns = [row[0] for row in sql_server_cursor]

            if snowflake_columns != sql_server_columns:
                text_box.insert(tk.END, f"Columns differ for table: {table}\n")
                text_box.insert(tk.END, f"Snowflake columns: {', '.join(snowflake_columns)}\n")
                text_box.insert(tk.END, f"SQL Server columns: {', '.join(sql_server_columns)}\n")
                text_box.insert(tk.END, "\n")
        else:
            text_box.insert(tk.END, f"Table missing in one of the databases: {table}\n\n")

    # Close connections
    snowflake_conn.close()
    sql_server_conn.close()

# Create a button to start the comparison
compare_button = tk.Button(window, text="Compare Tables", command=compare_tables)
compare_button.pack()

# Start the Tkinter event loop
window.mainloop()


In [ ]:
## THIS DOES NOT WORK AT ALL

import tkinter as tk
import snowflake.connector as sf
import pyodbc
from config import ssms_connector, account, user, authenticator, warehouse, database, role

# # Snowflake connection parameters
# SNOWFLAKE_ACCOUNT = 'your_snowflake_account'
# SNOWFLAKE_USER = 'your_snowflake_username'
# SNOWFLAKE_PASSWORD = 'your_snowflake_password'
# SNOWFLAKE_DATABASE = 'your_snowflake_database'

# # SQL Server connection parameters
# SQL_SERVER_DRIVER = '{SQL Server}'
# SQL_SERVER_SERVER = 'your_sql_server'
# SQL_SERVER_DATABASE = 'your_sql_server_database'
# SQL_SERVER_USER = 'your_sql_server_username'
# SQL_SERVER_PASSWORD = 'your_sql_server_password'

# Create a Tkinter GUI window
window = tk.Tk()
window.title("Table Comparison")
window.geometry("500x400")

# Create a text box to display the differences
text_box = tk.Text(window, height=20, width=60)
text_box.pack()

def compare_tables():
    # Clear the text box
    text_box.delete(1.0, tk.END)

    # Connect to Snowflake
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)

    # Connect to SQL Server
    sql_server_conn = pyodbc.connect(ssms_connector)

    # Get Snowflake schema names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SHOW SCHEMAS IN DATABASE {database}")
    snowflake_schemas = [row[1] for row in snowflake_cursor]

    # Compare tables across all schemas
    for schema in snowflake_schemas:
        # Get Snowflake table names
        snowflake_cursor.execute(f"SHOW TABLES IN {database}.{schema}")
        snowflake_tables = [row[1] for row in snowflake_cursor]

        # Get SQL Server table names
        sql_server_cursor = sql_server_conn.cursor()
        sql_server_cursor.execute(f"SELECT TABLE_NAME FROM GILBDW.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{schema}' AND TABLE_TYPE = 'BASE TABLE'")
        sql_server_tables = [row[0] for row in sql_server_cursor]

        # Compare tables
        for table in set(snowflake_tables + sql_server_tables):
            if table in snowflake_tables and table in sql_server_tables:
                # Compare table columns
                snowflake_cursor.execute(f"DESCRIBE TABLE {database}.{schema}.{table}")
                snowflake_columns = [row[1] for row in snowflake_cursor]
                sql_server_cursor.execute(f"SELECT COLUMN_NAME FROM GILBDW.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{schema}' AND TABLE_NAME = '{table}'")
                sql_server_columns = [row[0] for row in sql_server_cursor]

                if snowflake_columns != sql_server_columns:
                    text_box.insert(tk.END, f"Columns differ for table: {table} (Schema: {schema})\n")
                    text_box.insert(tk.END, f"Snowflake columns: {', '.join(snowflake_columns)}\n")
                    text_box.insert(tk.END, f"SQL Server columns: {', '.join(sql_server_columns)}\n")
                    text_box.insert(tk.END, "\n")
            else:
                text_box.insert(tk.END, f"Table missing in one of the databases: {table} (Schema: {schema})\n\n")

    # Close connections
    snowflake_conn.close()
    sql_server_conn.close()

# Create a button to start the comparison
compare_button = tk.Button(window, text="Compare Tables", command=compare_tables)
compare_button.pack()

# Start the Tkinter event loop
window.mainloop()


# New code to try

In [ ]:
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role
    )
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role
    )
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for i, (sf_row, ss_row) in enumerate(zip(snowflake_data, sql_server_data), start=1):
        if sf_row != ss_row:
            difference_reasons = []
            for sf_value, ss_value in zip(sf_row, ss_row):
                if sf_value != ss_value:
                    difference_reasons.append(f"Data value mismatch: {sf_value} != {ss_value}")
            differences.append((i, difference_reasons))

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(["Row Number", "Difference Reasons"])
            for diff_row in differences:
                csv_writer.writerow([diff_row[0], ", ".join(diff_row[1])])
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role
    )

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main event loop
window.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role
    )
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role
    )
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Get column names for Snowflake and SQL Server tables
    snowflake_columns = [desc[0] for desc in snowflake_cursor.description]
    sql_server_columns = [desc[0] for desc in sql_server_cursor.description]

    # Find the differences between the rows
    differences = []
    for i, (sf_row, ss_row) in enumerate(zip(snowflake_data, sql_server_data), start=1):
        if sf_row != ss_row:
            diff_reasons = []
            for j, (sf_value, ss_value) in enumerate(zip(sf_row, ss_row)):
                if sf_value != ss_value:
                    column_name = snowflake_columns[j] if j < len(snowflake_columns) else sql_server_columns[j]
                    diff_reasons.append((column_name, sf_value, ss_value))
            differences.append((i, diff_reasons))

    # Display the differences
    if differences:
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(["Row Number", "Column Name", "Snowflake Value", "SQL Server Value", "Reason"])
            for diff_row in differences:
                row_number = diff_row[0]
                for diff in diff_row[1]:
                    csv_writer.writerow([row_number, diff[0], diff[1], diff[2], "Data value mismatch"])
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role
    )

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main event loop
window.mainloop()

In [ ]:
#### DO NOT EDIT THIS CODE ####


# Import Dependencies
import tkinter as tk
from tkinter import ttk
import snowflake.connector as sf
import pyodbc
import csv
from config import ssms_connector, account, user, authenticator, warehouse, database, role 

# Get the Snowflake table names with schema
def get_snowflake_tables_with_schema():
    conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = []
    for row in cursor:
        schema_name = row[2]
        table_name = row[1]
        table_with_schema = f"{schema_name}.{table_name}"
        tables.append(table_with_schema)
    conn.close()
    return tables

# Get the SSMS table names
def get_sql_server_tables():
    conn_str = ssms_connector
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]
    conn.close()
    return tables

# Compare the data between the selected tables
def compare_tables():
    snowflake_table = snowflake_dropdown.get()
    sql_server_table = sql_server_table_var.get()

    # Display progress message
    progress_label.config(text="Comparing data...")
    window.update()

    # Snowflake data comparison
    snowflake_conn = sf.connect(
        account=account,
        user=user,
        authenticator=authenticator,
        warehouse=warehouse,
        database=database,
        role=role)
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute(f"SELECT * FROM {snowflake_table}")
    snowflake_data = snowflake_cursor.fetchall()

    # SQL Server data comparison
    sql_server_conn = pyodbc.connect(ssms_connector)
    sql_server_cursor = sql_server_conn.cursor()
    sql_server_cursor.execute(f"SELECT * FROM {sql_server_table}")
    sql_server_data = sql_server_cursor.fetchall()

    # Compare data between the selected tables
    differences = []
    for row in snowflake_data:
        if row not in sql_server_data:
            row_with_difference = ["Missing in SQL Server"] + list(row)
            differences.append(row_with_difference)

    for row in sql_server_data:
        if row not in snowflake_data:
            row_with_difference = ["Missing in Snowflake"] + list(row)
            differences.append(row_with_difference)

    # Display the differences
    if differences:
        header = ["Difference Reason"] + snowflake_cursor.description
        with open('Differences/differences.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(header)
            csv_writer.writerows(differences)
        print("Differences exported to differences.csv")
        # window.destroy()
    else:
        print("No Differences Found")

    # Update progress message
    progress_label.config(text="Comparison done.")
    window.update()

    snowflake_conn.close()
    sql_server_conn.close()

# Update the Snowflake tables dropdown
def update_snowflake_dropdown():
    # Establish Snowflake connection
    snowflake_conn = sf.connect(
        account = account,
        user = user,
        authenticator = authenticator,
        warehouse = warehouse,
        database = database,
        role = role)

    # Retrieve Snowflake schema and table names
    snowflake_cursor = snowflake_conn.cursor()
    snowflake_cursor.execute("SHOW TABLES")
    snowflake_tables = snowflake_cursor.fetchall()
    snowflake_table_names = [f"{row[3]}.{row[1]}" for row in snowflake_tables]

    # Update the Snowflake tables dropdown
    snowflake_dropdown['values'] = snowflake_table_names

    # Close the connection
    snowflake_conn.close()

# Create the main window
window = tk.Tk()
window.title("Table Data Comparison")
window.geometry("500x300")

# Create Snowflake tables dropdown
snowflake_label = tk.Label(window, text="Snowflake Tables:")
snowflake_label.pack()
snowflake_dropdown = ttk.Combobox(window)

# Initialize the Snowflake tables dropdown
update_snowflake_dropdown()  
snowflake_dropdown.pack()

# Create the SQL Server table drop-down list
sql_server_table_var = tk.StringVar()
sql_server_table_label = tk.Label(window, text="SQL Server Tables:")
sql_server_table_label.pack()
sql_server_table_dropdown = ttk.Combobox(window, textvariable=sql_server_table_var)
sql_server_table_dropdown['values'] = get_sql_server_tables()
sql_server_table_dropdown.pack()

# Create the Compare button
compare_button = tk.Button(window, text="Compare", command=compare_tables)
compare_button.pack(pady=10)

# Progress label
progress_label = tk.Label(window, text="", font=("Arial Bold", 10))
progress_label.pack(pady=10)

# Start the main even loop
window.mainloop()